In [1]:
import torch
import torch.nn as nn
import pdb

import os
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

/root/miniconda3/envs/myconda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load data
loader = torch.load('/mnt/fyp/data/val_loader.pt')
# device
device = torch.device("cuda:0")

In [30]:
model = torch.load('/mnt/fyp/models/teng_model.pth') # change to img_teng_imu / img / teng_imu / teng model

model.eval()

for idx,batch in enumerate(loader):
    with torch.no_grad():
        img_data_val = batch['data'][:,:224*224*3].reshape(-1,3,224,224).to(device)

        teng_data_val = batch['data'][:,224*224*3:224*224*3+500].reshape(-1,50,10).to(device) 

        imu_data_val = batch['data'][:,224*224*3+500:].reshape(-1,50,18).to(device) 

        label_data_val = batch['label'].to(torch.long).to(device) 
        label_data_val = label_data_val.squeeze()
        
        if idx==0:
            gt = label_data_val
        else:
            gt = torch.cat((gt, label_data_val), dim=0)

        #outputs_val = model(img_data_val, teng_data_val, imu_data_val)
        outputs_val = model(teng_data_val) # change according to model requirement

        _, predicted_val = torch.max(outputs_val, 1) 

        if idx==0:
            predicted = predicted_val
        else:
            predicted = torch.cat((predicted, predicted_val), dim=0)

In [43]:
right_num = [20 for i in range(39)] # 39*20
error = [[] for _ in range(39)] # 39*[]

In [44]:
res = (gt == predicted)

for i in range(res.shape[0]):
    if str(res[i].item()) == "False":
        right_num[gt[i].item()] -= 1
        error[gt[i].item()].append(predicted[i].item())

In [48]:
right_num = np.array(right_num)
np.save("/mnt/fyp/confusion_matrix/right_num.npy",right_num)
error = np.array(error)
np.save("/mnt/fyp/confusion_matrix/error.npy", error)

/tmp/ipykernel_106/1187661140.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  error = np.array(error)


In [2]:
import numpy as np
right_num = np.load("/mnt/fyp/confusion_matrix/right_num.npy")
print(right_num)

[11 19 19  1 15 14 16 20  0  0  0 10 17  1  0 10  6  1  0 16  0 17 12  0
 10  1  2  5  0  0  5  4 14  2 12 19  8  9 15]


In [6]:
error = np.load("/mnt/fyp/confusion_matrix/error.npy",allow_pickle=True)
print(error)

[list([6, 6, 21, 17, 36, 15, 36, 36, 36]) list([7]) list([31])
 list([34, 34, 27, 18, 33, 34, 33, 27, 34, 28, 34, 34, 34, 34, 34, 27, 16, 34, 34])
 list([19, 19, 21, 32, 19]) list([6, 36, 6, 6, 6, 6]) list([5, 0, 22, 5])
 list([])
 list([28, 34, 34, 27, 3, 33, 33, 27, 27, 3, 27, 3, 34, 34, 34, 27, 34, 34, 34, 34])
 list([33, 34, 33, 31, 27, 37, 27, 27, 33, 13, 11, 33, 33, 27, 29, 11, 34, 27, 13, 36])
 list([34, 36, 27, 27, 18, 34, 36, 34, 27, 27, 25, 34, 34, 34, 34, 36, 34, 34, 34, 34])
 list([17, 17, 17, 22, 16, 31, 22, 17, 17, 0]) list([1, 29, 11])
 list([3, 34, 8, 27, 3, 3, 33, 3, 33, 27, 34, 3, 3, 3, 28, 3, 33, 27, 33])
 list([34, 27, 10, 32, 37, 37, 27, 18, 26, 16, 18, 18, 33, 37, 17, 21, 18, 33, 3, 19])
 list([13, 18, 23, 18, 18, 23, 13, 26, 26, 13])
 list([31, 9, 15, 27, 31, 27, 31, 31, 26, 31, 31, 31, 26, 31])
 list([31, 2, 11, 2, 2, 2, 36, 31, 32, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
 list([27, 33, 27, 27, 27, 33, 33, 3, 8, 33, 3, 27, 27, 34, 27, 27, 29, 27, 33, 33])
 list([21, 4, 2